In [3]:
import numpy as np
import cv2
import time
import os

##from utils.grabscreen import grab_screen
##from utils.getkeys import key_check

In [4]:

import win32gui, win32ui, win32con, win32api

In [5]:
def grab_screen(region=None):

    hwin = win32gui.GetDesktopWindow()

    if region:
            left, top, x2, y2 = region
            width = x2 - left + 1
            height = y2 - top + 1
    else:
        width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
        height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
        left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
        top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc)
    memdc = srcdc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)
    
    signedIntsArray = bmp.GetBitmapBits(True)
    img = np.frombuffer(signedIntsArray, dtype='uint8')
    img.shape = (height, width, 4)

    #cv2.imwrite("imageFULL.png", img) 
    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())

    return img#cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

In [6]:
image = grab_screen(region=(0, 25, 800, 625))

# Region for top left window : region=(0, 25, 800, 625)

In [7]:
cv2.imwrite("imageFULL.png", image) 

True

In [8]:
## TEST THE OPENCV

In [9]:
#img = cv.imread(cv.samples.findFile("thisisfine.PNG"))
#if img is None:
#    sys.exit("Could not read the image.")
#cv.imshow("Display window", img)
#k = cv.waitKey(0)
#if k == ord("s"):
#    cv.imwrite("thisisfine2.PNG", img)

In [10]:
#!pip install XInput-Python
#!pip3 install pynput


In [11]:
import win32api as wapi
import XInput as XI

import time

In [12]:
XI.get_connected()

(True, True, True, False)

In [13]:
myState = XI.get_state(2)
XI.get_thumb_values(myState)
XI.get_button_values(myState)
#XI.get_trigger_values(myState)

{'DPAD_UP': False,
 'DPAD_DOWN': False,
 'DPAD_LEFT': False,
 'DPAD_RIGHT': False,
 'START': False,
 'BACK': False,
 'LEFT_THUMB': False,
 'RIGHT_THUMB': False,
 'LEFT_SHOULDER': False,
 'RIGHT_SHOULDER': False,
 'A': False,
 'B': False,
 'X': False,
 'Y': False}

In [14]:
def key_check():
    myState = XI.get_state(2)
    th_v = XI.get_thumb_values(myState)
    b_v = XI.get_button_values(myState)
    tr_v = XI.get_trigger_values(myState)
    
    return th_v, b_v, tr_v

In [15]:
#keyList = ["\b"]
#for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ 123456789,.'APS$/\\":
#    keyList.append(char)#
#
#def key_check():
#    keys = []
#    
#    keys.append(wapi.GetAsyncKeyState(wapi.VK_PAD_A))
#    
#    #for key in keyList:
#    #    if wapi.GetAsyncKeyState(ord(key)):
#    #        keys.append(key)
#    if 'H' in keys:
#        return 'H'
#    elif 'B' in keys:
#        return 'B'
#    elif 'A' in keys:
#        return 'A'
#    elif 'D' in keys:
#        return 'D'
#    elif ' ' in keys:
#        return ' '
#    else:
#        return 'W'

In [35]:

import cv2
import numpy as np 
import argparse
import time

def load_yolo():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
    #cv2.cuda.
    #net = cv2.dnn_DetectionModel(net)

    
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    return net, classes, colors, output_layers

In [36]:
def detect_objects(img, net, outputLayers):
    blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(outputLayers)
    return blob, outputs

In [37]:
def get_box_dimensions(outputs, height, width):
    boxes = []
    confs = []
    class_ids = []
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            #print(scores)
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > 0.3:
                center_x = int(detect[0] * width)
                center_y = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confs.append(float(conf))
                class_ids.append(class_id)
    return boxes, confs, class_ids

In [38]:
def draw_labels(boxes, confs, colors, class_ids, classes, img): 
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    return img

In [39]:
def image_detect(img_path): 
    model, classes, colors, output_layers = load_yolo()
    image, height, width, channels = load_image(img_path)
    blob, outputs = detect_objects(image, model, output_layers)
    boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
    draw_labels(boxes, confs, colors, class_ids, classes, image)
    
    
    #while True:
    #    key = cv2.waitKey(1)
    #    if key == 27:
    #        break

In [40]:
image_data = []
thumb_targets = []
button_targets = []
trigger_targets = []

model, classes, colors, output_layers = load_yolo()


count = 0
while True:
    count +=1
    last_time = time.time()
    image = grab_screen(region=(0, 25, 800, 625))

    
    image = cv2.resize(image, (400, 300))
    grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    
    height, width, channels = image.shape
    
    blob, outputs = detect_objects(image, model, output_layers)
    boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
    image = draw_labels(boxes, confs, colors, class_ids, classes, image)
    
    
    # Debug line to show image
    cv2.imshow("AI Peak", image)
    cv2.waitKey(1)

    # Convert to numpy array
    image = np.array(image)
    image_data.append(image)

    th_v, b_v, tr_v = key_check()
    thumb_targets.append(th_v)
    button_targets.append(b_v)
    trigger_targets.append(tr_v)
    
    if b_v.get('RIGHT_THUMB'):
        break

    print('loop took {} seconds'.format(time.time()-last_time))

AttributeError: 'cv2.dnn_DetectionModel' object has no attribute 'getLayerNames'

In [60]:
cv2.imwrite("imageFULL.png", image_data[-1]) 

True

In [80]:
#button_targets[-1]

In [ ]:
#th_v = XI.get_thumb_values(myState)


In [30]:
#image_data = []
#thumb_targets = []
#button_targets = []
#trigger_targets = []


#count = 0
#while True:
#    count +=1
#    last_time = time.time()
#    image = grab_screen(region=(0, 25, 800, 625))##
#
#    #kernel1 = np.array([[0, -1, 0],
#    #                    [-1, 5, -1],
#    #                    [0, -1, 0]])
#    #image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel1)##
#
#    #image = cv2.GaussianBlur(image, ksize=(5,5),sigmaX=0, sigmaY=0)    
#    
#    #image = cv2.Canny(image, threshold1=120, threshold2=150)
#    #image = cv2.
#    
#    image = cv2.resize(image, (400, 300))
#    grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)##
#
#    #circles = cv2.HoughCircles(grayImage,cv2.HOUGH_GRADIENT_ALT, 1, 20, param1=200, param2=0.8, minRadius=0, maxRadius=-1)
#    
#    #if circles is not None:
#     #   circles = np.uint16(circles)
#     #   for pt in circles[0, :]:
#     #       x, y, r = pt[0], pt[1], pt[2]
#     #       cv2.circle(image, (x,y), r, (0, 0, 255), 5)
#            
#    #image = cv2.circle(image, (x,y), radius=2, color=(0, 0, 0), thickness=-1)
#    #image = cv2.circle(image, (x,y), radius=2, color=(0, 0, 255), thickness=1)
#   # cv2.imshow(f"Circles", color)
#    
#    # Debug line to show image
#    cv2.imshow("AI Peak", image)
#    cv2.waitKey(1)
#
#    # Convert to numpy array
#    image = np.array(image)
#    image_data.append(image)#
#
#    th_v, b_v, tr_v = key_check()
#    thumb_targets.append(th_v)
#    button_targets.append(b_v)
#    trigger_targets.append(tr_v)
#    
#    if b_v.get('RIGHT_THUMB'):
#        break##
#
#    print('loop took {} seconds'.format(time.time()-last_time))